In [5]:
import numpy as np
import pandas as pd

data = np.load("NASDAQ_wiki_relation.npy")
ticker_wikicode_map_df = pd.read_csv("NASDAQ_ticker_to_wiki.csv")

In [10]:
import os

INDEX = "nasdaq100"

directory = "../../../data/" + INDEX + "/"

ticker_list = {}
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        sector, name, ticker, start, end = filename.split("-")
        ticker_list[ticker] = [name, sector]

In [14]:
wikicode_map = {}
for index, row in ticker_wikicode_map_df.iterrows():
    if row[0] in ticker_list:
        wikicode_map[row[0]] = row[1]

In [18]:
len(wikicode_map.values()), len(ticker_list.values())

(67, 102)

In [20]:
wikicode_map

{'AAPL': 'Q312',
 'ADBE': 'Q11463',
 'ADI': 'Q484930',
 'ADP': 'Q6394386',
 'ADSK': 'Q628051',
 'ALGN': 'unknown',
 'AMAT': 'Q7449678',
 'AMGN': 'Q470517',
 'AMZN': 'Q3884',
 'ANSS': 'Q295774',
 'ASML': 'Q297879',
 'ATVI': 'Q344341',
 'BIDU': 'Q14772',
 'BIIB': 'Q864338',
 'CDNS': 'Q608776',
 'CMCSA': 'Q1113804',
 'COST': 'Q715583',
 'CSCO': 'Q173395',
 'CSX': 'unknown',
 'CTAS': 'Q15109928',
 'CTSH': 'Q1107035',
 'DLTR': 'Q5289230',
 'DXCM': 'Q15109865',
 'EA': 'Q173941',
 'FAST': 'Q3067119',
 'FISV': 'Q1420513',
 'FTNT': 'Q2749364',
 'GILD': 'Q663596',
 'GOOG': 'Q95',
 'GOOGL': 'Q20800404',
 'IDXX': 'Q1758392',
 'ILMN': 'Q2068984',
 'INTC': 'Q248',
 'INTU': 'Q1318848',
 'ISRG': 'Q2743863',
 'KLAC': 'Q624649',
 'LRCX': 'Q1342041',
 'LULU': 'Q6702957',
 'MAR': 'Q1141173',
 'MCHP': 'Q1545076',
 'MDLZ': 'Q12857502',
 'MELI': 'Q950419',
 'MRVL': 'Q1347782',
 'MSFT': 'Q2283',
 'MU': 'Q1197548',
 'NFLX': 'Q907311',
 'NTES': 'Q196259',
 'NVDA': 'Q182477',
 'NXPI': 'Q1155668',
 'ODFL': 'Q7083

In [19]:
for ticker, val in ticker_list.items():
    if ticker not in wikicode_map:
        print(ticker)

CPRT
KDP
DDOG
SIRI
DOCU
META
BKNG
CRWD
CEG
ZM
ENPH
PANW
XEL
MTCH
TEAM
AVGO
MRNA
AEP
LCID
AMD
EBAY
ABNB
PEP
PDD
MNST
KHC
JD
ZS
WBA
PYPL
HON
CHTR
TMUS
EXC
AZN


In [37]:
import pywikibot

#Get the item
site = pywikibot.Site("wikidata", "wikidata")
repo = site.data_repository()
item = pywikibot.ItemPage(repo, "Q3884")

In [38]:
item_dict = item.get()

In [39]:
item_dict.keys()

dict_keys(['labels', 'descriptions', 'aliases', 'claims', 'sitelinks'])

In [40]:
for k, v in item_dict['claims'].items():
    print(k)
    break

P1297


In [42]:
item_dict['claims']['P355']

[Claim.fromJSON(DataSite("wikidata", "wikidata"), {'mainsnak': {'snaktype': 'value', 'property': 'P355', 'datatype': 'wikibase-item', 'datavalue': {'value': {'entity-type': 'item', 'numeric-id': 280457}, 'type': 'wikibase-entityid'}}, 'type': 'statement', 'id': 'Q3884$FE88D0E3-0BEE-47C5-BFD5-1B4CBB2EB3FD', 'rank': 'normal'}),
 Claim.fromJSON(DataSite("wikidata", "wikidata"), {'mainsnak': {'snaktype': 'value', 'property': 'P355', 'datatype': 'wikibase-item', 'datavalue': {'value': {'entity-type': 'item', 'numeric-id': 456157}, 'type': 'wikibase-entityid'}}, 'type': 'statement', 'id': 'Q3884$E3D62E03-7DE2-4A86-811F-B3D9D5BE2076', 'rank': 'normal', 'qualifiers': {'P1107': [{'snaktype': 'value', 'property': 'P1107', 'datatype': 'quantity', 'datavalue': {'value': {'amount': '+1.0', 'upperBound': None, 'lowerBound': None, 'unit': '1'}, 'type': 'quantity'}, 'hash': '339f06569cdb61b426750370aca6b841f9155a43'}]}, 'qualifiers-order': ['P1107'], 'references': [{'snaks': {'P248': [{'snaktype': 'va

In [17]:
import json

x = open('NASDAQ_connections.json', "r")
# parse x:
y = json.loads(x.read())

In [20]:
y['Q5969562']

{'Q20022777': [['P17', 'P17'], ['P31', 'P31'], ['P414', 'P414']],
 'Q6648755': [['P31', 'P31'], ['P414', 'P414']],
 'Q470517': [['P17', 'P17'], ['P31', 'P31'], ['P414', 'P414']],
 'Q4816705': [['P17', 'P17'], ['P414', 'P414']],
 'Q5969568': [['P17', 'P17'], ['P31', 'P31'], ['P414', 'P414']],
 'Q1045842': [['P414', 'P414']],
 'Q1544413': [['P17', 'P17'], ['P31', 'P31'], ['P414', 'P414']],
 'Q5001347': [['P17', 'P17'], ['P31', 'P31'], ['P414', 'P414']],
 'Q18156655': [['P17', 'P17'], ['P414', 'P414']],
 'Q17183462': [['P17', 'P17'], ['P31', 'P31'], ['P414', 'P414']],
 'Q1071853': [['P17', 'P17'], ['P414', 'P414']],
 'Q6702957': [['P31', 'P31'], ['P414', 'P414']],
 'Q297879': [['P31', 'P31'], ['P414', 'P414']],
 'Q431454': [['P31', 'P31'], ['P414', 'P414']],
 'Q946080': [['P31', 'P31'], ['P414', 'P414']],
 'Q1325369': [['P31', 'P31'], ['P414', 'P414']],
 'Q20800404': [['P17', 'P17'], ['P31', 'P31'], ['P414', 'P414']],
 'Q6843476': [['P31', 'P31'], ['P414', 'P414']],
 'Q1665429': [['P17', 